### DATA Transformation

In [1]:
1+1

2

In [2]:
import os 
%pwd

'c:\\Fythonn\\fyt\\Main\\Project_4\\research'

In [ ]:

os.chdir("../")

In [4]:
%pwd

'c:\\Fythonn\\fyt\\Main\\Project_4'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataTransformationConfig:
    root_dir: Path
    datadir: Path
    tokenizer_name: Path

    

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
 

In [7]:
from venv import create
from pyparsing import C


class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_filepaths = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepaths)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            datadir = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config

In [9]:
import os 
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self, examples):
        input_encodings = self.tokenizer(examples["dialogue"], truncation=True, padding="max_length", max_length=512)

        with self.tokenizer.as_target_tokenizer():
             target_encodings = self.tokenizer(examples["summary"], truncation=True, padding="max_length", max_length=150)

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.datadir)
        dataset_samsum_pt= dataset_samsum.map(self.convert_examples_to_features, batched=True)
        return dataset_samsum_pt

In [12]:
config = ConfigurationManager()
data_tranformation_config= config.get_data_transformation_config()
data_tranformation = DataTransformation(data_tranformation_config)
data_tranformation.convert()

[2025-02-26 14:02:06,504: INFO: common: yaml file: c:\Fythonn\fyt\Main\Project_4\config\config.yaml loaded successfully]
[2025-02-26 14:02:06,510: INFO: common: yaml file: c:\Fythonn\fyt\Main\Project_4\params.yaml loaded successfully]
[2025-02-26 14:02:06,514: INFO: common: created directory at: artifacts]
[2025-02-26 14:02:06,519: INFO: common: created directory at: artifacts/data_transformation]


c:\Fythonn\fyt\Main\Project_4\venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Fythonn\fyt\Main\Project_4\venv\lib\site-packages

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})